# Investments results inspection
Routines for converting, inspecting and visualizing _pommesinvest_ investment results.
* Plot investment decisions
* Plot decomissionings
* Plot total installed capacity
* Include exogenous capacities to evaluate plants status

## Package imports

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pommesevaluation.investment_results_inspection import (
    preprocess_raw_results, 
    aggregate_investment_results, 
    extract_data_and_save,
    plot_single_investment_variable, 
    plot_single_investment_variable_for_all_cases,
    create_single_plot,
    plot_sensitivities,
    plot_generation_and_consumption_for_all_cases,
)
from pommesevaluation.global_vars import (
    FUELS, 
    FUELS_RENAMED, 
    STORAGES, 
    STORAGES_RENAMED,
    STORAGES_OTHER,
    STORAGES_OTHER_RENAMED,
    DEMAND_RESPONSE, 
    DEMAND_RESPONSE_RENAMED, 
    ELECTROLYZER, 
    ELECTROLYZER_RENAMED
)
from pommesevaluation.tools import update_matplotlib_params

## Script parameterization
* Define model configuration / results to be analyzed
* Define column order and colors for evaluation
* Read in the actual results

In [ ]:
# Global settings
LANGUAGE = "German"  # "German", "English"
PLOT_CONFIG = {
    "single": {
        "figsize": (14, 8),
        "bbox": (0.5, -0.2),
    },
    "scenarios": {
        "fig_width": 14,
        "subplot_height": 4,
        "fig_position": 0.08,
        "fig_position_low": 0.05,
        "fig_position_high": 0.12,
        "bbox": (0.5, -0.35),
    },
    "sensitivities": {
        "fig_width": 14,
        "subplot_height": 4,
        "fig_position": 0.08,
        "fig_position_low": 0.03,
        "fig_position_high": 0.12,
        "bbox": (0.5, -0.35),
    }
}
exclude = True  # Exclude unchosen fuels if True

# Model configuration
time_frame_in_years = 26
frequency = "1H"
dr_scenario = "50"
dr_scenarios = ["none", "5", "50", "95"]
sensitivities = {
    "pv": ["pv_-50%", "pv_-25%", "", "pv_+25%", "pv_+50%"],
    "prices": ["prices_-50%", "prices_-25%", "", "prices_+25%", "prices_+50%"],
    "interconnection": ["interconnection_-50%", "interconnection_-25%", "", "interconnection_+25%", "interconnection_+50%"]
}
fuel_price_scenario = "NZE"
emissions_pathway = "long-term"
impose_investment_maxima = False

# Paths, filenames and color codes
path_results = "./model_results/pommesinvest/"
path_processed_data = "./data_out/"
path_plots = "./plots/"

path_inputs = "./model_inputs/pommesinvest/"

sensitivities_rename_dict = {
    "German": {
        "pv": "PV-Erzeugung",
        "prices": "Brennstoff und $CO_{2}$-Preise",
        "interconnection": "Interkonnektorenkapazitäten"
    },
    "English": {
        "pv": "pv generation",
        "prices": "fuel and $CO_{2}$ prices",
        "interconnection": "interconnector capacities"
    },
}
potential_col_names = {
    "German": {
        "percent utilized": "prozentualer Anteil",
        "realised": "realisiert",
        "potential": "Potenzial",
    },
    "English": {
        "percent utilized": "percent utilized",
        "realised": "realised",
        "potential": "potential",
    }
}

SENSITIVITY_COLORS = {
    "pv": "#fcb001",
    "prices": "#809cfd",
    "interconnection": "#838383",
}
FUELS_EXISTING = {
    f: FUELS[f] 
    for f in FUELS 
    if f in ["uranium", "lignite", "hardcoal", "mixedfuels", "otherfossil"]
}

FUELS_NEW = {
    f: FUELS[f] 
    for f in FUELS 
    if f in ["biomass", "hydrogen", "natgas", "oil", "waste"]
}

STORAGES_INFLOW = {
    f"{stor}_inflow": STORAGES[stor] for stor in STORAGES
}

DR_TOTAL = {
    "German": "Lastmanagement gesamt",
    "English": "demand reponse overall",
}

PLOT_TITLE = {
    "German": "Szenario DR",
    "English": "scenario DR",
}
PLOT_TITLE_ADD = {
    "German": "; Sensitivität:",
    "English": "; sensitivity:",
}

filename = (
    f"investment_LP_start-2020-01-01_{time_frame_in_years}"
    f"-years_simple_freq_{frequency}"
)
if impose_investment_maxima:
    annual_investment_limits = ""
else:
    annual_investment_limits = "_no_annual_limit"
if dr_scenario != "none":
    file_add_on = (
        f"_with_dr_{dr_scenario}_"
        f"fuel_price-{fuel_price_scenario}_"
        f"co2_price-{emissions_pathway}{annual_investment_limits}_investment"
    )
else:
    file_add_on = (
        f"_no_dr_50_"
        f"fuel_price-{fuel_price_scenario}_"
        f"co2_price-{emissions_pathway}{annual_investment_limits}_investment"
    )
file_extension = ".csv"

filename_backup = f"installed_capacity_backup_generation_{LANGUAGE}.csv"
filename_storage = f"installed_storage_capacity_{LANGUAGE}.csv"

filename_demand_response_potentials = {
    dr_scen: [
        f"{dr_cluster}_potential_parameters_{dr_scen}%.csv"
        for dr_cluster in DEMAND_RESPONSE
    ] for dr_scen in dr_scenarios if dr_scen != "none" 
}

# Control workflow
include_storages_in_plots = True
aggregation_mode = "energy_carrier"

In [ ]:
update_matplotlib_params(
    small_size=14, medium_size=16, large_size=18
)

# Single scenario analyses
Inspect the results for a single scenario model run.
## Read in, preprocess and aggregate data
See imported functions for documentation

In [ ]:
investment_results_raw = pd.read_csv(f"{path_results}{filename}{file_add_on}{file_extension}", index_col=0)
processed_results = preprocess_raw_results(investment_results_raw)
aggregated_results, other_storage_results = aggregate_investment_results(
    processed_results, energy_carriers=FUELS, by="energy_carrier"
)
aggregated_results.rename(
    index={
        **FUELS_RENAMED[LANGUAGE], 
        **{f"{stor}_inflow": STORAGES_RENAMED[LANGUAGE][stor] for stor in STORAGES_RENAMED[LANGUAGE]}, 
        **DEMAND_RESPONSE_RENAMED[LANGUAGE], 
        **ELECTROLYZER_RENAMED[LANGUAGE]
    }, 
    inplace=True
)
other_storage_results.rename(
    index={
        **STORAGES_OTHER_RENAMED[LANGUAGE]
    },
    inplace=True
)

## Visualize results
Results visualization is separated between investments in storage energy content as well as storage outflow power and the remainder, i.e. investments in generation units, investments in storage inflow power and investments in demand response clusters.

The following parameters are displayed
* Newly installed capacity
* Decommissioned capacity
* Total installed capacity

### Generation, storage inflow and demand response

In [ ]:
if include_storages_in_plots:
    colors = {
            **{FUELS_RENAMED[LANGUAGE][f]: FUELS_NEW[f] for f in FUELS_RENAMED[LANGUAGE] if f in FUELS_NEW}, 
            **{STORAGES_RENAMED[LANGUAGE][s]: STORAGES[s] for s in STORAGES_RENAMED[LANGUAGE]},
            **{ELECTROLYZER_RENAMED[LANGUAGE][e]: ELECTROLYZER[e] for e in ELECTROLYZER_RENAMED[LANGUAGE]},
        }
    if dr_scenario != "none":
        colors = {
            **colors,
            **{DEMAND_RESPONSE_RENAMED[LANGUAGE][d]: DEMAND_RESPONSE[d] for d in DEMAND_RESPONSE_RENAMED[LANGUAGE]}
        }
else:
    colors = {
        **{FUELS_RENAMED[LANGUAGE][f]: FUELS_NEW[f] for f in FUELS_RENAMED[LANGUAGE] if f in FUELS_NEW}, 
    }
    if dr_scenario != "none":
        colors = {
                **colors,
                **{DEMAND_RESPONSE_RENAMED[LANGUAGE][d]: DEMAND_RESPONSE[d] for d in DEMAND_RESPONSE_RENAMED[LANGUAGE]}
            }

plot_single_investment_variable(
    aggregated_results,
    "invest",
    colors=colors,
    aggregation=aggregation_mode,
    save=True,
    filename="results_invest_dr_scenario",
    dr_scenario=dr_scenario,
    path_plots=path_plots,
    path_data_out=path_processed_data,
    ylim=[0, 30000],
    language=LANGUAGE,
    figsize=PLOT_CONFIG["single"]["figsize"],
    bbox_params=PLOT_CONFIG["single"]["bbox"],
)

In [ ]:
if include_storages_in_plots:
    colors = {
            **{FUELS_RENAMED[LANGUAGE][f]: FUELS_NEW[f] for f in FUELS_RENAMED[LANGUAGE] if f in FUELS_NEW}, 
            **{STORAGES_RENAMED[LANGUAGE][s]: STORAGES[s] for s in STORAGES_RENAMED[LANGUAGE]},
            **{ELECTROLYZER_RENAMED[LANGUAGE][e]: ELECTROLYZER[e] for e in ELECTROLYZER_RENAMED[LANGUAGE]},
        }
    if dr_scenario != "none":
        colors = {
            **colors,
            **{DEMAND_RESPONSE_RENAMED[LANGUAGE][d]: DEMAND_RESPONSE[d] for d in DEMAND_RESPONSE_RENAMED[LANGUAGE]}
        }
else:
    colors = {
        **{FUELS_RENAMED[LANGUAGE][f]: FUELS_NEW[f] for f in FUELS_RENAMED[LANGUAGE] if f in FUELS_NEW}, 
    }
    if dr_scenario != "none":
        colors = {
                **colors,
                **{DEMAND_RESPONSE_RENAMED[LANGUAGE][d]: DEMAND_RESPONSE[d] for d in DEMAND_RESPONSE_RENAMED[LANGUAGE]}
            }

plot_single_investment_variable(
    aggregated_results,
    "old",
    colors=colors,
    aggregation=aggregation_mode,
    save=True,
    filename="results_old_dr_scenario",
    dr_scenario=dr_scenario,
    path_plots=path_plots,
    path_data_out=path_processed_data,
    ylim=[0, 30000],
    language=LANGUAGE,
    figsize=PLOT_CONFIG["single"]["figsize"],
    bbox_params=PLOT_CONFIG["single"]["bbox"],
)

In [ ]:
if include_storages_in_plots:
    colors = {
            **{FUELS_RENAMED[LANGUAGE][f]: FUELS_NEW[f] for f in FUELS_RENAMED[LANGUAGE] if f in FUELS_NEW}, 
            **{STORAGES_RENAMED[LANGUAGE][s]: STORAGES[s] for s in STORAGES_RENAMED[LANGUAGE]},
            **{ELECTROLYZER_RENAMED[LANGUAGE][e]: ELECTROLYZER[e] for e in ELECTROLYZER_RENAMED[LANGUAGE]},
        }
    if dr_scenario != "none":
        colors = {
            **colors,
            **{DEMAND_RESPONSE_RENAMED[LANGUAGE][d]: DEMAND_RESPONSE[d] for d in DEMAND_RESPONSE_RENAMED[LANGUAGE]}
        }
else:
    colors = {
        **{FUELS_RENAMED[LANGUAGE][f]: FUELS_NEW[f] for f in FUELS_RENAMED[LANGUAGE] if f in FUELS_NEW}, 
    }
    if dr_scenario != "none":
        colors = {
                **colors,
                **{DEMAND_RESPONSE_RENAMED[LANGUAGE][d]: DEMAND_RESPONSE[d] for d in DEMAND_RESPONSE_RENAMED[LANGUAGE]}
            }

plot_single_investment_variable(
    aggregated_results,
    "total",
    colors=colors,
    aggregation=aggregation_mode,
    save=True,
    filename="results_total_dr_scenario",
    dr_scenario=dr_scenario,
    path_plots=path_plots,
    path_data_out=path_processed_data,
    ylim=[0, 150000],
    language=LANGUAGE,
    figsize=PLOT_CONFIG["single"]["figsize"],
    bbox_params=PLOT_CONFIG["single"]["bbox"],
)

### Storage outflow and storage energy content

In [ ]:
storages_colors = {
    STORAGES_OTHER_RENAMED[LANGUAGE][s]: STORAGES_OTHER[s] 
    for s in STORAGES_OTHER_RENAMED[LANGUAGE]
}
plot_single_investment_variable(
    other_storage_results,
    "invest",
    colors=storages_colors,
    aggregation="energy_carrier",
    storage=True,
    save=True,
    filename="results_storages_invest_dr_scenario",
    dr_scenario=dr_scenario,
    path_plots=path_plots,
    path_data_out=path_processed_data,
    ylim=[0, 30000],
    language=LANGUAGE,
    figsize=PLOT_CONFIG["single"]["figsize"],
    bbox_params=PLOT_CONFIG["single"]["bbox"],
)

In [ ]:
plot_single_investment_variable(
    other_storage_results,
    "old",
    colors=storages_colors,
    aggregation="energy_carrier",
    storage=True,
    save=True,
    filename="results_storages_old_dr_scenario",
    dr_scenario=dr_scenario,
    path_plots=path_plots,
    path_data_out=path_processed_data,
    ylim=[0, 30000],
    language=LANGUAGE,
    figsize=PLOT_CONFIG["single"]["figsize"],
    bbox_params=PLOT_CONFIG["single"]["bbox"],
)

In [ ]:
plot_single_investment_variable(
    other_storage_results,
    "total",
    colors=storages_colors,
    aggregation="energy_carrier",
    storage=True,
    save=True,
    filename="results_storages_total_dr_scenario",
    dr_scenario=dr_scenario,
    path_plots=path_plots,
    path_data_out=path_processed_data,
    ylim=[0, 150000],
    language=LANGUAGE,
    figsize=PLOT_CONFIG["single"]["figsize"],
    bbox_params=PLOT_CONFIG["single"]["bbox"],
)

## Include exogenous capacities in assessment
* Read in capacities of backup plants, aggregated by fuel
* Read in installed power and capacity of storage units

In [ ]:
backup_generation = pd.read_csv(
    f"{path_processed_data}{filename_backup}", index_col=0
)
if time_frame_in_years <= 26:
    backup_generation = backup_generation.loc[:2045]
backup_generation.index = backup_generation.index.astype(str)

In [ ]:
grouped_total = aggregated_results[["total"]].reset_index()
grouped_total = grouped_total.pivot(
    index="energy_carrier", columns="year", values="total"
).T

In [ ]:
overall_capacity = grouped_total.add(backup_generation)
overall_capacity.loc[:,overall_capacity.isna().all()] = backup_generation
overall_capacity.loc[:,overall_capacity.isna().all()] = grouped_total

In [ ]:
if include_storages_in_plots:
    colors = {
            **{FUELS_RENAMED[LANGUAGE][f]: FUELS[f] for f in FUELS_RENAMED[LANGUAGE] if f in FUELS},
            **{FUELS_RENAMED[LANGUAGE][f]: FUELS_NEW[f] for f in FUELS_RENAMED[LANGUAGE] if f in FUELS_NEW},
            **{STORAGES_RENAMED[LANGUAGE][s]: STORAGES[s] for s in STORAGES_RENAMED[LANGUAGE]},
            **{ELECTROLYZER_RENAMED[LANGUAGE][e]: ELECTROLYZER[e] for e in ELECTROLYZER_RENAMED[LANGUAGE]},
        }
    if dr_scenario != "none":
        colors = {
            **colors,
            **{DEMAND_RESPONSE_RENAMED[LANGUAGE][d]: DEMAND_RESPONSE[d] for d in DEMAND_RESPONSE_RENAMED[LANGUAGE]}
        }
else:
    colors = {
        **{FUELS_RENAMED[LANGUAGE][f]: FUELS_NEW[f] for f in FUELS_RENAMED[LANGUAGE] if f in FUELS_NEW}, 
    }
    if dr_scenario != "none":
        colors = {
                **colors,
                **{DEMAND_RESPONSE_RENAMED[LANGUAGE][d]: DEMAND_RESPONSE[d] for d in DEMAND_RESPONSE_RENAMED[LANGUAGE]}
            }

plot_single_investment_variable(
    overall_capacity.T,
    "all",
    colors=colors,
    group=False,
    save=True,
    filename="results_all_dr_scenario",
    dr_scenario=dr_scenario,
    path_plots=path_plots,
    path_data_out=path_processed_data,
    ylim=[0, 200000],
    language=LANGUAGE,
    figsize=PLOT_CONFIG["single"]["figsize"],
    bbox_params=PLOT_CONFIG["single"]["bbox"],
)

In [ ]:
if dr_scenario != "none":
    overall_capacity[DR_TOTAL[LANGUAGE]] = overall_capacity[[
        val for val in DEMAND_RESPONSE_RENAMED[LANGUAGE].values()
    ]].sum(axis=1)

In [ ]:
overall_capacity

# Multi scenario analyses
Inspect and compare the results among scenarios.

## Read in, preprocess and aggregate data

In [ ]:
investment_results_raw = {}
aggregated_results = {}
other_storage_results = {}
for dr_scenario in dr_scenarios:
    if dr_scenario != "none":
        file_add_on = (
            f"_with_dr_{dr_scenario}_"
            f"fuel_price-{fuel_price_scenario}_"
            f"co2_price-{emissions_pathway}{annual_investment_limits}_investment"
        )
    else:
        file_add_on = (
            f"_no_dr_50_"
            f"fuel_price-{fuel_price_scenario}_"
            f"co2_price-{emissions_pathway}{annual_investment_limits}_investment"
        )
    investment_results_raw[dr_scenario] = (
        pd.read_csv(f"{path_results}{filename}{file_add_on}{file_extension}", index_col=0)
    )
    processed_results = preprocess_raw_results(investment_results_raw[dr_scenario])
    aggregated_results[dr_scenario], other_storage_results[dr_scenario] = aggregate_investment_results(
        processed_results, energy_carriers=FUELS, by="energy_carrier"
    )
    aggregated_results[dr_scenario].rename(
        index={
            **FUELS_RENAMED[LANGUAGE], 
            **{f"{stor}_inflow": STORAGES_RENAMED[LANGUAGE][stor] for stor in STORAGES_RENAMED[LANGUAGE]}, 
            **DEMAND_RESPONSE_RENAMED[LANGUAGE], 
            **ELECTROLYZER_RENAMED[LANGUAGE]
        },
        inplace=True
    )
    other_storage_results[dr_scenario].rename(
        index={
            **STORAGES_OTHER_RENAMED[LANGUAGE]
        },
        inplace=True
    )   

## Visualize results
### Generation, storage inflow and demand response

In [ ]:
if exclude:
    to_exclude = ["oil", "waste", "biomass"]
    aggregated_results_manipulated = aggregated_results.copy()
    for key in aggregated_results_manipulated:
        aggregated_results_manipulated[key] = aggregated_results_manipulated[key].loc[~aggregated_results_manipulated[key].index.get_level_values(0).isin([FUELS_RENAMED[LANGUAGE][f] for f in to_exclude])]
else:
    to_exclude = []
    aggregated_results_manipulated = aggregated_results.copy()

if include_storages_in_plots:
    colors = {
        **{FUELS_RENAMED[LANGUAGE][f]: FUELS_NEW[f] for f in FUELS_RENAMED[LANGUAGE] if f in FUELS_NEW}, 
        **{STORAGES_RENAMED[LANGUAGE][s]: STORAGES[s] for s in STORAGES_RENAMED[LANGUAGE]},
        **{ELECTROLYZER_RENAMED[LANGUAGE][e]: ELECTROLYZER[e] for e in ELECTROLYZER_RENAMED[LANGUAGE]},
        **{DEMAND_RESPONSE_RENAMED[LANGUAGE][d]: DEMAND_RESPONSE[d] for d in DEMAND_RESPONSE_RENAMED[LANGUAGE]},
    }
else:
    colors = {
        **{FUELS_RENAMED[LANGUAGE][f]: FUELS_NEW[f] for f in FUELS_RENAMED[LANGUAGE] if f in FUELS_NEW}, 
        **{DEMAND_RESPONSE_RENAMED[LANGUAGE][d]: DEMAND_RESPONSE[d] for d in DEMAND_RESPONSE_RENAMED[LANGUAGE]}
    }

dr_color_codes = {
    DEMAND_RESPONSE_RENAMED[LANGUAGE][d]: DEMAND_RESPONSE[d] 
    for d in DEMAND_RESPONSE_RENAMED[LANGUAGE]
}
colors[FUELS_RENAMED[LANGUAGE]["natgas"]] = "#ceaf51" 

plot_single_investment_variable_for_all_cases(
    aggregated_results_manipulated,
    "invest",
    colors=colors,
    aggregation="energy_carrier",
    dr_color_codes=dr_color_codes,
    save=True,
    filename="results_invest",
    path_plots=path_plots,
    ylim=(0, 30000),
    title=PLOT_TITLE[LANGUAGE],
    language=LANGUAGE,
    bbox_params=PLOT_CONFIG["scenarios"]["bbox"],
    fig_width=PLOT_CONFIG["scenarios"]["fig_width"],
    subplot_height=PLOT_CONFIG["scenarios"]["subplot_height"],
    fig_position=PLOT_CONFIG["scenarios"]["fig_position"],
    grid=True,
    edges=True,
)

In [ ]:
plot_single_investment_variable_for_all_cases(
    aggregated_results,
    "old",
    colors=colors,
    aggregation="energy_carrier",
    dr_color_codes=dr_color_codes,
    save=True,
    filename="results_old",
    path_plots=path_plots,
    ylim=(0, 30000),
    title=PLOT_TITLE[LANGUAGE],
    language=LANGUAGE,
    bbox_params=PLOT_CONFIG["scenarios"]["bbox"],
    fig_width=PLOT_CONFIG["scenarios"]["fig_width"],
    subplot_height=PLOT_CONFIG["scenarios"]["subplot_height"],
    fig_position=PLOT_CONFIG["scenarios"]["fig_position"],
)

In [ ]:
plot_single_investment_variable_for_all_cases(
    aggregated_results,
    "total",
    colors=colors,
    aggregation="energy_carrier",
    dr_color_codes=dr_color_codes,
    save=True,
    filename="results_total",
    path_plots=path_plots,
    ylim=(0, 150000),
    title=PLOT_TITLE[LANGUAGE],
    language=LANGUAGE,
    bbox_params=PLOT_CONFIG["scenarios"]["bbox"],
    fig_width=PLOT_CONFIG["scenarios"]["fig_width"],
    subplot_height=PLOT_CONFIG["scenarios"]["subplot_height"],
    fig_position=PLOT_CONFIG["scenarios"]["fig_position"],
)

### Storage outflow and storage energy content

In [ ]:
storages_colors = {
    STORAGES_OTHER_RENAMED[LANGUAGE][s]: STORAGES_OTHER[s] 
    for s in STORAGES_OTHER_RENAMED[LANGUAGE]
}
plot_single_investment_variable_for_all_cases(
    other_storage_results,
    "invest",
    colors=storages_colors,
    aggregation="energy_carrier",
    storage=True,
    dr_color_codes={},
    save=True,
    filename="results_storages_invest",
    path_plots=path_plots,
    ylim=(0, 30000),
    title=PLOT_TITLE[LANGUAGE],
    language=LANGUAGE,
    bbox_params=PLOT_CONFIG["scenarios"]["bbox"],
    fig_width=PLOT_CONFIG["scenarios"]["fig_width"],
    subplot_height=PLOT_CONFIG["scenarios"]["subplot_height"],
    fig_position=PLOT_CONFIG["scenarios"]["fig_position_low"],
)

In [ ]:
plot_single_investment_variable_for_all_cases(
    other_storage_results,
    "old",
    colors=storages_colors,
    aggregation="energy_carrier",
    storage=True,
    dr_color_codes={},
    save=True,
    filename="results_storages_old",
    path_plots=path_plots,
    ylim=(0, 30000),
    title=PLOT_TITLE[LANGUAGE],
    language=LANGUAGE,
    bbox_params=PLOT_CONFIG["scenarios"]["bbox"],
    fig_width=PLOT_CONFIG["scenarios"]["fig_width"],
    subplot_height=PLOT_CONFIG["scenarios"]["subplot_height"],
    fig_position=PLOT_CONFIG["scenarios"]["fig_position_low"],
)

In [ ]:
plot_single_investment_variable_for_all_cases(
    other_storage_results,
    "total",
    colors=storages_colors,
    aggregation="energy_carrier",
    storage=True,
    dr_color_codes={},
    save=True,
    filename="results_storages_total",
    path_plots=path_plots,
    ylim=(0, 150000),
    title=PLOT_TITLE[LANGUAGE],
    language=LANGUAGE,
    bbox_params=PLOT_CONFIG["scenarios"]["bbox"],
    fig_width=PLOT_CONFIG["scenarios"]["fig_width"],
    subplot_height=PLOT_CONFIG["scenarios"]["subplot_height"],
    fig_position=PLOT_CONFIG["scenarios"]["fig_position_low"],
)

## Include exogenous capacities in assessment

In [ ]:
try:
    backup_generation
except NameError:
    backup_generation = pd.read_csv(
        f"{path_processed_data}{filename_backup}", index_col=0
    )
    if time_frame_in_years <= 26:
        backup_generation = backup_generation.loc[:2045]
    backup_generation.index = backup_generation.index.astype(str)

overall_capacity = {}
for dr_scenario in dr_scenarios:
    grouped_total = aggregated_results[dr_scenario][["total"]].reset_index()
    grouped_total = grouped_total.pivot(
        index="energy_carrier", columns="year", values="total"
    ).T
    overall_capacity[dr_scenario] = grouped_total.add(backup_generation)
    overall_capacity[dr_scenario].loc[:, overall_capacity[dr_scenario].isna().all()] = backup_generation
    overall_capacity[dr_scenario].loc[:, overall_capacity[dr_scenario].isna().all()] = grouped_total
    overall_capacity[dr_scenario] = overall_capacity[dr_scenario].T

In [ ]:
if include_storages_in_plots:
    colors = {
        **{FUELS_RENAMED[LANGUAGE][f]: FUELS[f] for f in FUELS_RENAMED[LANGUAGE] if f in FUELS},
        **{FUELS_RENAMED[LANGUAGE][f]: FUELS_NEW[f] for f in FUELS_RENAMED[LANGUAGE] if f in FUELS_NEW},
        **{STORAGES_RENAMED[LANGUAGE][s]: STORAGES[s] for s in STORAGES_RENAMED[LANGUAGE]},
        **{ELECTROLYZER_RENAMED[LANGUAGE][e]: ELECTROLYZER[e] for e in ELECTROLYZER_RENAMED[LANGUAGE]},
        **{DEMAND_RESPONSE_RENAMED[LANGUAGE][d]: DEMAND_RESPONSE[d] for d in DEMAND_RESPONSE_RENAMED[LANGUAGE]},
    }
else:
    colors = {
        **{FUELS_RENAMED[LANGUAGE][f]: FUELS[f] for f in FUELS_RENAMED[LANGUAGE] if f in FUELS},
        **{FUELS_RENAMED[LANGUAGE][f]: FUELS_NEW[f] for f in FUELS_RENAMED[LANGUAGE] if f in FUELS_NEW},
        **{DEMAND_RESPONSE_RENAMED[LANGUAGE][d]: DEMAND_RESPONSE[d] for d in DEMAND_RESPONSE_RENAMED[LANGUAGE]}
    }
colors[FUELS_RENAMED[LANGUAGE]["natgas"]] = "#ceaf51" 
    
plot_single_investment_variable_for_all_cases(
    overall_capacity,
    "all",
    colors=colors,
    aggregation="energy_carrier",
    dr_color_codes=dr_color_codes,
    group=False,
    save=True,
    filename="results_all",
    path_plots=path_plots,
    ylim=(0, 200000),
    title=PLOT_TITLE[LANGUAGE],
    language=LANGUAGE,
    bbox_params=PLOT_CONFIG["scenarios"]["bbox"],
    fig_width=PLOT_CONFIG["scenarios"]["fig_width"],
    subplot_height=PLOT_CONFIG["scenarios"]["subplot_height"],
    fig_position=PLOT_CONFIG["scenarios"]["fig_position_high"],
    grid=True,
    edges=True,
)

In [ ]:
for dr_scenario in dr_scenarios:
    if dr_scenario != "none":
        overall_capacity[dr_scenario].loc[DR_TOTAL[LANGUAGE]] = overall_capacity[dr_scenario].loc[[
            val for val in DEMAND_RESPONSE_RENAMED[LANGUAGE].values()
        ]].sum()
    overall_capacity[dr_scenario]["max"] = overall_capacity[dr_scenario].max(axis=1)
    overall_capacity[dr_scenario].to_csv(
        f"{path_processed_data}overall_capacity_dr_scenario_{dr_scenario}.csv", sep=";"
    )

## Compare available vs. utilized demand response potential (in terms of investments)

In [ ]:
annual_capacity_limits = {}
overall_capacity_limits = {}
for dr_scenario in dr_scenarios:
    if dr_scenario != "none":
        annual_limit = pd.DataFrame()
        for entry in filename_demand_response_potentials[dr_scenario]:
            params = pd.read_csv(f"{path_inputs}{entry}", index_col=0)
            annual_maximum = [
                min(
                    max(
                        params.loc[
                            iter_year, "potential_pos_overall"
                        ],
                        params.loc[
                            iter_year, "potential_neg_overall"
                        ],
                    ),
                    params.loc[iter_year, "max_cap"],
                )
                for iter_year in range(
                    2020, 2020 + time_frame_in_years
                )
            ]
            annual_limit[entry.rsplit("_", 3)[0]] = annual_maximum
        annual_capacity_limits[dr_scenario] = annual_limit
        annual_limit.rename(columns=DEMAND_RESPONSE_RENAMED[LANGUAGE], inplace=True)
        potential_vs_installed = pd.DataFrame()
        potential_vs_installed["potential"] = annual_limit.max()
        potential_vs_installed["realised"] = overall_capacity[dr_scenario].loc[list(DEMAND_RESPONSE_RENAMED[LANGUAGE].values()), "max"]
        if LANGUAGE == "German":
            potential_vs_installed.columns = ["Potenzial", "realisiert"]
        overall_capacity_limits[dr_scenario] = potential_vs_installed

In [ ]:
plot_single_investment_variable(
    overall_capacity_limits[dr_scenario],
    "potential",
    colors=dr_color_codes,
    storage=False,
    group=False,
    save=False,
    filename="potential",
    dr_scenario="none",
    format_axis=False,
    draw_xlabel=False,
    language=LANGUAGE,
    ncol=3,
    bbox_params=(0.5, -0.2),
)

In [ ]:
plot_single_investment_variable_for_all_cases(
    overall_capacity_limits,
    "potential",
    colors=dr_color_codes,
    storage=False,
    group=False,
    save=True,
    filename="potential",
    title=f"{PLOT_TITLE[LANGUAGE]}",
    format_axis=False,
    include_common_xlabel=False,
    fig_width=8,
    subplot_height=4,
    ncol=2,
    bbox_params=(0.5, -0.5),
    language=LANGUAGE,
    edges=True,
)

In [ ]:
for dr_scenario in dr_scenarios:
    if dr_scenario != "none":
        overall_capacity_limits[dr_scenario][potential_col_names[LANGUAGE]["percent utilized"]] = (
            overall_capacity_limits[dr_scenario][potential_col_names[LANGUAGE]["realised"]] 
            / overall_capacity_limits[dr_scenario][potential_col_names[LANGUAGE]["potential"]]
            * 100
        )
        overall_capacity_limits[dr_scenario].to_csv(
            f"{path_processed_data}potential_utilization_dr_scenario_{dr_scenario}.csv", sep=";"
        )

# Analyses of sensitivities
Inspect and compare among the sensitivities considered for scenario DR 50

## Read in, preprocess and aggregate data

In [ ]:
aggregated_results = {}
other_storage_results = {}
file_add_on = (
    f"_with_dr_50_"
    f"fuel_price-{fuel_price_scenario}_"
    f"co2_price-{emissions_pathway}{annual_investment_limits}_"
)
renamed_sensitivities = {}

for sensitivity_item, sensitivity_values in sensitivities.items():
    renamed_sensitivities[sensitivity_item] = dict()
    aggregated_results[sensitivity_item] = dict()
    other_storage_results[sensitivity_item] = dict()
    renamed_sensitivities[sensitivity_item] = []
    for sensitivity_value in sensitivity_values:
        if sensitivity_value != "":
            new_name = sensitivities_rename_dict[LANGUAGE][sensitivity_value.split('_')[0]]
            renamed_sensitivity = f"{new_name} {sensitivity_value.split('_')[-1]}"
            renamed_sensitivities[sensitivity_item].append(renamed_sensitivity)
            file_add_on_sensitivity = f"sensitivity_{sensitivity_value}_investment"
        else:
            # results from standard scenario
            file_add_on_sensitivity = "investment"
            renamed_sensitivity = f"{sensitivities_rename_dict[LANGUAGE][sensitivity_item]} +/-0"

        investment_results_raw = (
            pd.read_csv(f"{path_results}{filename}{file_add_on}{file_add_on_sensitivity}{file_extension}", index_col=0)
        )
        processed_results = preprocess_raw_results(investment_results_raw)
        aggregated_results[sensitivity_item][renamed_sensitivity], other_storage_results[sensitivity_item][renamed_sensitivity] = aggregate_investment_results(
            processed_results, energy_carriers=FUELS, by="energy_carrier"
        )
        aggregated_results[sensitivity_item][renamed_sensitivity].rename(
            index={
                **FUELS_RENAMED[LANGUAGE], 
                **{f"{stor}_inflow": STORAGES_RENAMED[LANGUAGE][stor] for stor in STORAGES_RENAMED[LANGUAGE]}, 
                **DEMAND_RESPONSE_RENAMED[LANGUAGE], 
                **ELECTROLYZER_RENAMED[LANGUAGE]
            },
            inplace=True
        )
        other_storage_results[sensitivity_item][renamed_sensitivity].rename(
            index={
                **STORAGES_OTHER_RENAMED[LANGUAGE]
            },
            inplace=True
        )

## Visualize results
### Generation, storage inflow and demand response

In [ ]:
if include_storages_in_plots:
    colors = {
        **{FUELS_RENAMED[LANGUAGE][f]: FUELS_NEW[f] for f in FUELS_RENAMED[LANGUAGE] if f in FUELS_NEW}, 
        **{STORAGES_RENAMED[LANGUAGE][s]: STORAGES[s] for s in STORAGES_RENAMED[LANGUAGE]},
        **{ELECTROLYZER_RENAMED[LANGUAGE][e]: ELECTROLYZER[e] for e in ELECTROLYZER_RENAMED[LANGUAGE]},
        **{DEMAND_RESPONSE_RENAMED[LANGUAGE][d]: DEMAND_RESPONSE[d] for d in DEMAND_RESPONSE_RENAMED[LANGUAGE]},
    }
else:
    colors = {
        **{FUELS_RENAMED[LANGUAGE][f]: FUELS_NEW[f] for f in FUELS_RENAMED[LANGUAGE] if f in FUELS_NEW}, 
        **{DEMAND_RESPONSE_RENAMED[LANGUAGE][d]: DEMAND_RESPONSE[d] for d in DEMAND_RESPONSE_RENAMED[LANGUAGE]}
    }

dr_color_codes = {
    DEMAND_RESPONSE_RENAMED[LANGUAGE][d]: DEMAND_RESPONSE[d] 
    for d in DEMAND_RESPONSE_RENAMED[LANGUAGE]
}

for sensitivity_item in sensitivities:
    plot_single_investment_variable_for_all_cases(
        aggregated_results[sensitivity_item],
        "invest",
        colors=colors,
        aggregation="energy_carrier",
        dr_color_codes=dr_color_codes,
        save=True,
        filename="results_invest_sensitivities",
        path_plots=path_plots,
        ylim=(0, 50000),
        title=f"{PLOT_TITLE[LANGUAGE]} 50{PLOT_TITLE_ADD[LANGUAGE]}",
        language=LANGUAGE,
        bbox_params=PLOT_CONFIG["sensitivities"]["bbox"],
        fig_width=PLOT_CONFIG["sensitivities"]["fig_width"],
        subplot_height=PLOT_CONFIG["sensitivities"]["subplot_height"],
        fig_position=PLOT_CONFIG["sensitivities"]["fig_position"],
    )

In [ ]:
for sensitivity_item in sensitivities:
    plot_single_investment_variable_for_all_cases(
        aggregated_results[sensitivity_item],
        "old",
        colors=colors,
        aggregation="energy_carrier",
        dr_color_codes=dr_color_codes,
        save=True,
        filename="results_old_sensitivities",
        path_plots=path_plots,
        ylim=(0, 30000),
        title=f"{PLOT_TITLE[LANGUAGE]} 50{PLOT_TITLE_ADD[LANGUAGE]}",
        language=LANGUAGE,
        bbox_params=PLOT_CONFIG["sensitivities"]["bbox"],
        fig_width=PLOT_CONFIG["sensitivities"]["fig_width"],
        subplot_height=PLOT_CONFIG["sensitivities"]["subplot_height"],
        fig_position=PLOT_CONFIG["sensitivities"]["fig_position"],
    )

In [ ]:
for sensitivity_item in sensitivities:
    plot_single_investment_variable_for_all_cases(
        aggregated_results[sensitivity_item],
        "total",
        colors=colors,
        aggregation="energy_carrier",
        dr_color_codes=dr_color_codes,
        save=True,
        filename="results_total_sensitivities",
        path_plots=path_plots,
        ylim=(0, 200000),
        title=f"{PLOT_TITLE[LANGUAGE]} 50{PLOT_TITLE_ADD[LANGUAGE]}",
        language=LANGUAGE,
        bbox_params=PLOT_CONFIG["sensitivities"]["bbox"],
        fig_width=PLOT_CONFIG["sensitivities"]["fig_width"],
        subplot_height=PLOT_CONFIG["sensitivities"]["subplot_height"],
        fig_position=PLOT_CONFIG["sensitivities"]["fig_position"],
    )

### Storage outflow and storage energy content

In [ ]:
storages_colors = {
    STORAGES_OTHER_RENAMED[LANGUAGE][s]: STORAGES_OTHER[s] 
    for s in STORAGES_OTHER_RENAMED[LANGUAGE]
}
for sensitivity_item in sensitivities:
    plot_single_investment_variable_for_all_cases(
        other_storage_results[sensitivity_item],
        "invest",
        colors=storages_colors,
        aggregation="energy_carrier",
        storage=True,
        dr_color_codes={},
        save=True,
        filename="results_storages_invest_sensitivities",
        path_plots=path_plots,
        ylim=(0, 30000),
        title=f"{PLOT_TITLE[LANGUAGE]} 50{PLOT_TITLE_ADD[LANGUAGE]}",
        language=LANGUAGE,
        bbox_params=PLOT_CONFIG["sensitivities"]["bbox"],
        fig_width=PLOT_CONFIG["sensitivities"]["fig_width"],
        subplot_height=PLOT_CONFIG["sensitivities"]["subplot_height"],
        fig_position=PLOT_CONFIG["sensitivities"]["fig_position_low"],
    )

In [ ]:
for sensitivity_item in sensitivities:
    plot_single_investment_variable_for_all_cases(
        other_storage_results[sensitivity_item],
        "old",
        colors=storages_colors,
        aggregation="energy_carrier",
        storage=True,
        dr_color_codes={},
        save=True,
        filename="results_storages_old_sensitivities",
        path_plots=path_plots,
        ylim=(0, 30000),
        title=f"{PLOT_TITLE[LANGUAGE]} 50{PLOT_TITLE_ADD[LANGUAGE]}",
        language=LANGUAGE,
        bbox_params=PLOT_CONFIG["sensitivities"]["bbox"],
        fig_width=PLOT_CONFIG["sensitivities"]["fig_width"],
        subplot_height=PLOT_CONFIG["sensitivities"]["subplot_height"],
        fig_position=PLOT_CONFIG["sensitivities"]["fig_position_low"],
    )

In [ ]:
for sensitivity_item in sensitivities:
    plot_single_investment_variable_for_all_cases(
        other_storage_results[sensitivity_item],
        "total",
        colors=storages_colors,
        aggregation="energy_carrier",
        storage=True,
        dr_color_codes={},
        save=True,
        filename="results_storages_total_sensitivities",
        path_plots=path_plots,
        ylim=(0, 200000),
        title=f"{PLOT_TITLE[LANGUAGE]} 50{PLOT_TITLE_ADD[LANGUAGE]}",
        language=LANGUAGE,
        bbox_params=PLOT_CONFIG["sensitivities"]["bbox"],
        fig_width=PLOT_CONFIG["sensitivities"]["fig_width"],
        subplot_height=PLOT_CONFIG["sensitivities"]["subplot_height"],
        fig_position=PLOT_CONFIG["sensitivities"]["fig_position_low"],
    )

## Include exogenous capacities in assessment

In [ ]:
# Check if has been declared earlier (depends on code execution)
try:
    overall_capacity
except NameError:
    overall_capacity = dict()
    
try:
    backup_generation
except NameError:
    backup_generation = pd.read_csv(
        f"{path_processed_data}{filename_backup}", index_col=0
    )
    if time_frame_in_years <= 26:
        backup_generation = backup_generation.loc[:2045]
    backup_generation.index = backup_generation.index.astype(str)

for sensitivity_item, sensitivity_values in renamed_sensitivities.items():
    overall_capacity[sensitivity_item] = dict()
    for sensitivity_value in sensitivity_values:
        grouped_total = aggregated_results[sensitivity_item][sensitivity_value][["total"]].reset_index()
        grouped_total = grouped_total.pivot(
            index="energy_carrier", columns="year", values="total"
        ).T
        overall_capacity[sensitivity_item][sensitivity_value] = grouped_total.add(backup_generation)
        overall_capacity[sensitivity_item][sensitivity_value].loc[:, overall_capacity[sensitivity_item][sensitivity_value].isna().all()] = backup_generation
        overall_capacity[sensitivity_item][sensitivity_value].loc[:, overall_capacity[sensitivity_item][sensitivity_value].isna().all()] = grouped_total
        overall_capacity[sensitivity_item][sensitivity_value] = overall_capacity[sensitivity_item][sensitivity_value].T

In [ ]:
if include_storages_in_plots:
    colors = {
        **{FUELS_RENAMED[LANGUAGE][f]: FUELS[f] for f in FUELS_RENAMED[LANGUAGE] if f in FUELS},
        **{FUELS_RENAMED[LANGUAGE][f]: FUELS_NEW[f] for f in FUELS_RENAMED[LANGUAGE] if f in FUELS_NEW},
        **{STORAGES_RENAMED[LANGUAGE][s]: STORAGES[s] for s in STORAGES_RENAMED[LANGUAGE]},
        **{ELECTROLYZER_RENAMED[LANGUAGE][e]: ELECTROLYZER[e] for e in ELECTROLYZER_RENAMED[LANGUAGE]},
        **{DEMAND_RESPONSE_RENAMED[LANGUAGE][d]: DEMAND_RESPONSE[d] for d in DEMAND_RESPONSE_RENAMED[LANGUAGE]},
    }
else:
    colors = {
        **{FUELS_RENAMED[LANGUAGE][f]: FUELS[f] for f in FUELS_RENAMED[LANGUAGE] if f in FUELS},
        **{FUELS_RENAMED[LANGUAGE][f]: FUELS_NEW[f] for f in FUELS_RENAMED[LANGUAGE] if f in FUELS_NEW},
        **{DEMAND_RESPONSE_RENAMED[LANGUAGE][d]: DEMAND_RESPONSE[d] for d in DEMAND_RESPONSE_RENAMED[LANGUAGE]}
    }

for sensitivity_item in sensitivities:
    plot_single_investment_variable_for_all_cases(
        overall_capacity[sensitivity_item],
        "all",
        colors=colors,
        aggregation="energy_carrier",
        dr_color_codes=dr_color_codes,
        group=False,
        save=True,
        filename="results_all_sensitivities",
        path_plots=path_plots,
        ylim=(0, 350000),
        title=f"{PLOT_TITLE[LANGUAGE]} 50{PLOT_TITLE_ADD[LANGUAGE]}",
        language=LANGUAGE,
        bbox_params=PLOT_CONFIG["sensitivities"]["bbox"],
        fig_width=PLOT_CONFIG["sensitivities"]["fig_width"],
        subplot_height=PLOT_CONFIG["sensitivities"]["subplot_height"],
        fig_position=PLOT_CONFIG["sensitivities"]["fig_position_high"],
    )

In [ ]:
for sensitivity_item, sensitivity_values in renamed_sensitivities.items():
    for number, sensitivity_value in enumerate(sensitivity_values):
        overall_capacity[sensitivity_item][sensitivity_value].loc[DR_TOTAL[LANGUAGE]] = overall_capacity[sensitivity_item][sensitivity_value].loc[[
            val for val in DEMAND_RESPONSE_RENAMED[LANGUAGE].values()
        ]].sum()
        overall_capacity[sensitivity_item][sensitivity_value]["max"] = overall_capacity[sensitivity_item][sensitivity_value].max(axis=1)
        original_name = sensitivities[sensitivity_item][number]
        if original_name != "":
            overall_capacity[sensitivity_item][sensitivity_value].to_csv(
                f"{path_processed_data}overall_capacity_dr_scenario_50_sensitivity_{original_name.split('_')[0]}_{original_name.split('_')[1]}.csv", 
                sep=";"
            )

## Visualize sensitivities
Introduce rather "classical" line charts to display the sensitivitiy towards changes of input parameters:
* Create a joint data set with maximum overall capacities per technology
* Create subplots to visualize the sensitivities

In [ ]:
max_capacities_per_kind = dict()
cols_to_consider = [FUELS_RENAMED[LANGUAGE]["hydrogen"], STORAGES_RENAMED[LANGUAGE]["battery"], DR_TOTAL[LANGUAGE]]
for sensitivity_item, sensitivity_values in renamed_sensitivities.items():
    max_capacities_per_kind[sensitivity_item] = pd.DataFrame()
    for number, sensitivity_value in enumerate(sensitivity_values):
        if number == 2:
            # Insert base case
            max_capacities_per_kind[sensitivity_item].loc["+/-0", cols_to_consider] = overall_capacity["50"].loc[cols_to_consider, "max"]
        max_capacities_per_kind[sensitivity_item].loc[sensitivity_value.split(" ")[-1], cols_to_consider] = overall_capacity[
            sensitivity_item
        ][sensitivity_value].loc[cols_to_consider, "max"]

In [ ]:
colors = {
    **{FUELS_RENAMED[LANGUAGE][f]: FUELS_NEW[f] for f in FUELS_RENAMED[LANGUAGE] if f in FUELS_NEW},
    **{STORAGES_RENAMED[LANGUAGE][s]: STORAGES[s] for s in STORAGES_RENAMED[LANGUAGE]},
    DR_TOTAL[LANGUAGE]: "k"
}
plot_sensitivities(
    max_capacities_per_kind,
    colors,
    renamed_sensitivities,
    language=LANGUAGE,
    label_used="x_axis",
    filename="sensitivities_by_kind",
)

In [ ]:
max_capacities_per_tech = dict()
for col in cols_to_consider:
    max_capacities_per_tech[col] = pd.DataFrame()
    for sensitivity_item, sensitivity_values in renamed_sensitivities.items():
        for number, sensitivity_value in enumerate(sensitivity_values):
            if number == 2:
                # Insert base case
                max_capacities_per_tech[col].loc["+/-0", sensitivities_rename_dict[LANGUAGE][sensitivity_item]] = overall_capacity["50"].loc[col, "max"]
            max_capacities_per_tech[col].loc[sensitivity_value.split(" ")[-1], sensitivities_rename_dict[LANGUAGE][sensitivity_item]] = overall_capacity[
                sensitivity_item
            ][sensitivity_value].loc[col, "max"]

In [ ]:
colors = {
    sensitivities_rename_dict[LANGUAGE][s]: SENSITIVITY_COLORS[s] for s in SENSITIVITY_COLORS
}
plot_sensitivities(
    max_capacities_per_tech,
    colors,
    language=LANGUAGE,
    label_used="title",
    filename="sensitivities_by_tech",
)

## Compare available vs. utilized demand response potential (in terms of investments)

In [ ]:
for sensitivity_item, sensitivity_values in renamed_sensitivities.items():
    overall_capacity_limits[sensitivity_item] = dict()
    for number, sensitivity_value in enumerate(sensitivity_values):
        potential_vs_installed = pd.DataFrame()
        potential_vs_installed["potential"] = annual_capacity_limits["50"].max()
        potential_vs_installed["realised"] = overall_capacity[sensitivity_item][sensitivity_value].loc[list(DEMAND_RESPONSE_RENAMED[LANGUAGE].values()), "max"]
        if LANGUAGE == "German":
            potential_vs_installed.columns = ["Potenzial", "realisiert"]
        overall_capacity_limits[sensitivity_item][sensitivity_value] = potential_vs_installed 

In [ ]:
to_plot = {}
for sensitivity_item in sensitivities:
    to_plot[sensitivity_item] = {}
    for number, (key, val) in enumerate(overall_capacity_limits[sensitivity_item].items()):
        if number == 0:
            to_plot[sensitivity_item]["Potenzial"] = val[["Potenzial"]].T
        to_plot[sensitivity_item][key] = val[["realisiert"]].T

In [ ]:
for sensitivity_item in sensitivities:
    # HACK: Use slightly extended existing method for results visualization
    plot_generation_and_consumption_for_all_cases(
        to_plot[sensitivity_item],
        "",
        0,
        colors=dr_color_codes,
        fig_height=6,
        subplot_width=4,
        save=True,
        path_plots="./plots/",
        filename=f"potential_sensitivity_comparion_{sensitivity_item}",
        place_legend_below=True,
        ncol=4,
        bbox_params=(-2.5, -0.45),
        language="German",
        wspace=0.5,
        y_label_pos=(0.04, 0.5),
        format_axis=True,
        hide_legend_and_xlabel=False,
        x_slices=(5, 16),
        slice_time=False,
        sharey=True,
        edges=True,
    )

In [ ]:
for sensitivity_item, sensitivity_values in renamed_sensitivities.items():
    for number, sensitivity_value in enumerate(sensitivity_values):
        overall_capacity_limits[sensitivity_item][sensitivity_value][potential_col_names[LANGUAGE]["percent utilized"]] = (
            overall_capacity_limits[sensitivity_item][sensitivity_value][potential_col_names[LANGUAGE]["realised"]] 
            / overall_capacity_limits[sensitivity_item][sensitivity_value][potential_col_names[LANGUAGE]["potential"]]
            * 100
        )
        if number >= 2:
            original_name = sensitivities[sensitivity_item][number + 1]
        else:
            original_name = sensitivities[sensitivity_item][number]
        if original_name != "":
            overall_capacity_limits[sensitivity_item][sensitivity_value].to_csv(
                f"{path_processed_data}potential_utilization_dr50_sensitivity_{original_name.split('_')[0]}_{original_name.split('_')[1]}.csv", 
                sep=";"
            )

# Preparation of results for usage in AMIRIS
> _**Note**:_
>
> * _The following section creates the necessary outputs that can be passed to the AMIRIS converter notebook._
> * _Data is extracted and saved but not visualized._
> * _Data aggregation differs compared to the remainder of this notebook._

Extract data for all scenarios and save respective outputs to file.

In [ ]:
for dr_scen in dr_scenarios:
    if dr_scen != "none":
        file_add_on = (
            f"_with_dr_{dr_scen}_"
            f"fuel_price-{fuel_price_scenario}_"
            f"co2_price-{emissions_pathway}{annual_investment_limits}_investment"
        )
    else:
        file_add_on = (
            f"_no_dr_50_"
            f"fuel_price-{fuel_price_scenario}_"
            f"co2_price-{emissions_pathway}{annual_investment_limits}_investment"
        )
    investment_results_raw = (
        pd.read_csv(f"{path_results}{filename}{file_add_on}{file_extension}", index_col=0)
    )
    processed_results = preprocess_raw_results(investment_results_raw)
    aggregated_results, other_storage_results = aggregate_investment_results(
        processed_results, 
        energy_carriers=FUELS, 
        by=["energy_carrier", "technology"], 
        include_chp_information=True
    )    
    
    for variable_name in ["invest", "old", "total"]:
        extract_data_and_save(
            aggregated_results,
            variable_name,
            aggregation=["energy_carrier", "technology"],
            group=True,
            filename=f"results_for_amiris_{variable_name}_dr_scenario",
            dr_scenario=dr_scen,
            path_data_out="./data_out/",
        )